In [1]:
# necessary imports
import networkx as nx
import math
import random

global T0
global G

In [2]:
# temparature function at iteration k
def temperatureSchedule(k):
  alpha = 0.01
  Tk = (1-(alpha*k))*T0
  return Tk

In [3]:
# value function
def valueFun(G):
  inFlowD = 0
  # total inflows into destination node
  for (x,y) in G.edges():
      if(y==4): # as destination node is 9
        inFlowD += G.edges[x,y]['flow']
  return(inFlowD)

In [5]:
# calculate nearest neighbor solution
def nearestNeighbor(G):
  newG = G.copy()
  check = True
  # select random path
  path = next(nx.all_simple_paths(G, source=3, target=4))
  # loop until no vioaltion occurs along P
  while(check):
    check = False
    # change weights by +/-3000
    positive = True
    for i in range(len(path)-1):
          eweights = G.edges[path[i],path[i+1]]['flow']+3000
          # check for Capacity constraints
          if(eweights>G.edges[path[i],path[i+1]]['capacity']):
            positive = False
            break
          newG.edges[path[i],path[i+1]]['flow'] = eweights
          
    if(positive == False):
      for i in range(len(path)-1):
          eweights = G.edges[path[i],path[i+1]]['flow']-3000
          newG.edges[path[i],path[i+1]]['flow'] = eweights
    # check law of conservation for path P
    for i in range(len(path)-2):
      if(newG.edges[path[i],path[i+1]]['flow'] != newG.edges[path[i+1],path[i+2]]['flow']):
         check = True
  return newG

In [6]:
# Function for simulated annealing
def SimulatedAnnealing(G):
  k=1
  # repeat from k=1 to some N
  while(k>0):
    if(k==1):
      T = T0
    else:
      T = temperatureSchedule(k)
    # if T is 0 then return the solution
    if(T==0):
      return G
    # if not, calculate G'(new Graph), nearest neighbor of G
    new_G = nearestNeighbor(G)
    delE = valueFun(new_G)-valueFun(G)
    if(delE>0):
      G = new_G
    else:
      # calculate probability
      V = prob = math.exp(delE/T) 
      E = (int(math.log10(V)))-1 # exponent
      M = V / 10**E # mantissa
      # generate random number using M and E
      rN = random.randint(1,pow(10,abs(E)))
      # check if number is acceptable
      if(rN>=1 and rN<=M):
        G = new_G
        # else discard new graph 
    #and continue to next iteration
    k+=1
  return G 

In [4]:
# check for law of conservation for each node
def checkConservationLaw(newG):
  for n in newG.nodes():
    inFlow = 0
    outFlow = 0
    for (x,y) in newG.edges():
      if(y==n):
        inFlow += newG.edges[x,y]['flow']
      elif(x==n):
        outFlow += newG.edges[x,y]['flow']
    if(inFlow==outFlow):
      continue
    else:
      return False
  return True

In [7]:
def initialSol(G):
   # repeat until law of conservation obeys
    while(checkConservationLaw(G)==False):
      for (u, v) in G.edges():
        capacity = G.edges[u,v]['capacity']
        flow = G.edges[u,v]['flow'] = random.randint(4000,10000)
        # check if flow is less than/equal to capacity, if not reassign
        while(flow>capacity):
          flow = G.edges[u,v]['flow'] = random.randint(4000,10000)
    return G

In [8]:
#set initial temperature to 10k
T0 = 10000

# Generate random graph
seed=1000  
G= nx.gnp_random_graph (50, .3, seed=seed) # Graph with nodes 50 and connectivity 3

# Check if graph is connected
if(nx.is_connected(G)):
    for (u, v) in G.edges():
      # assign capacity to each edge
      capacity = G.edges[u,v]['capacity'] = random.randint(6000,12000)
      # assign flow to each edge
      flow = G.edges[u,v]['flow'] = random.randint(4000,10000)
      # check if flow is less than/equal to capacity, if not reassign
      while(flow>capacity):
        flow = G.edges[u,v]['flow'] = random.randint(4000,10000)
     
# calculate initial solution     
G = initialSol(G)

# Call simulated annealing function with the generated graph 
soln = SimulatedAnnealing(G)

maxFlow = valueFun(soln)
print(maxFlow)

KeyboardInterrupt: ignored

In [10]:
# max flow value using Edmonds Karp algorithm from Networkx
from networkx.algorithms.flow import edmonds_karp
R = edmonds_karp(G, 3, 4)
flow_value = nx.maximum_flow_value(G, 3, 4)
print(flow_value)

132350


4030